## Ingest
Bring .csv into the system

In [1]:
from ingest import load_file
import os
os.listdir('assets/data')

['dws_wages.csv',
 'usbe_students.csv',
 'ushe_students.csv',
 'ustc_students.csv']

In [2]:
load_file('assets/data/dws_wages.csv', 'dws_wages')
load_file('assets/data/usbe_students.csv', 'usbe_students')
load_file('assets/data/ustc_students.csv', 'ustc_students') 
load_file('assets/data/ushe_students.csv', 'ushe_students')

## Linking Steps

Process of Record Linkage

0. Load
1. Preprocess
2. Index
3. Compare
4. Classify
5. Evaluate
6. Update (Post MPI - Adding new information, new MPI, to MPI Master Record)

### Master Person Architectures
Visualize data architecture store of MPI raw records.  NOTE: data may not be present in all database options.  Use whichever cell aligns with configured loading behavior.

**SQL** View

In [1]:
from db import get_session
import pandas as pd 


with get_session() as session:
    result = session.execute(
        'SELECT * FROM mpi_vectors LIMIT 5'
    ).fetchall()
    count_mpi = session.execute(
        'SELECT COUNT(*) FROM mpi_vectors'
    ).fetchone()
pd.DataFrame(result)

,0,1,2,3
0,1,None,elvis,11
1,1,presley,elvis,12
2,1,costello,elvis,13
3,2,austin,jane,11
4,2,austin,janet,12


In [2]:
print('Total MPI in system: ', str(count_mpi[0]))

Total MPI in system:  5


**NoSQL** View

In [1]:
from db import get_mongo
import json

db = get_mongo()
count_docs = db.raw.count_documents({})
x = db.raw.find_one({})
x['_id'] = str(x['_id'])

print(json.dumps(x, indent=2))

TypeError: 'NoneType' object is not subscriptable

In [2]:
# Delete all records (skip if not needed)
d = db.raw.delete_many({})
d.deleted_count

10000

In [3]:
print('Total MPI in system: ', count_docs)

Total MPI in system:  10000


## Prepare Data

Prepare identity view (MPI vectors) and data view (distinct mapped columns from source)

In [1]:
source_tablename = 'usbe_students'
# source_tablename = 'ustc_students'
# source_tablename = 'ushe_students'
# source_tablename = 'dws_wages'

In [2]:
from mpi.prepare import create_data_view, create_identity_view

dview = create_data_view(source_tablename)
iview = create_identity_view(mapped_columns=dview.subset.columns)

In [3]:
dview.head()

,BIRTH_DATE,FIRST_ENTERED_US,FIRST_NAME,GENDER,LAST_NAME,MIDDLE_NAME,SSID,STUDENT_ID,id,guid
0,6/9/1256,07-MAR-16 12.00.00.000000000 AM,Genora,f,Zegarra,Rhino,0x218b2ea30x43165d46,666165,46394,1489953705965142162
1,9/5/1287,13-AUG-12 12.00.00.000000000 AM,Laramee,m,Vela Garcia,Legacy-Martin,0x2156f0e80x42ade1d0,976990,35263,1489953705965142162
2,3/11/1507,06-OCT-16 12.00.00.000000000 AM,Zelma,n,Garlitz,Katempa,0x2981558d0x5302ab1a,225275,148149,1489953705965142162
3,3/11/1898,24-FEB-97 12.00.00.000000000 AM,Rynale,n,Reuter,Javien,0x175401eb0x2ea803d6,445502272,126536,1489953705965142162
4,1/5/2380,05-SEP-05 12.00.00.000000000 AM,Weslee,n,Cavalcante,Milam,0x286b652b0x50d6ca56,437170,105139,1489953705965142162


In [4]:
dview.subset.head()

,birth_date_pool,first_name_pool,gender_pool,last_name_pool,middle_name_pool,ssid_pool,usbe_student_id_pool,guid
0,6/9/1256,genora,f,zegarra,rhino,0x218b2ea30x43165d46,666165,1489953705965142162
1,9/5/1287,laramee,m,vela garcia,legacy martin,0x2156f0e80x42ade1d0,976990,1489953705965142162
2,3/11/1507,zelma,n,garlitz,katempa,0x2981558d0x5302ab1a,225275,1489953705965142162
3,3/11/1898,rynale,n,reuter,javien,0x175401eb0x2ea803d6,445502272,1489953705965142162
4,1/5/2380,weslee,n,cavalcante,milam,0x286b652b0x50d6ca56,437170,1489953705965142162


In [5]:
iview.head()

,first_name_pool,mpi,birth_date_pool,gender_pool,last_name_pool,ssid_pool,middle_name_pool,freq_score
0,None,1,None,None,None,None,None,0.57735
3,genora,15551146-12008325-6004855-1758529,6/9/1256,f,zegarra,0x218b2ea30x43165d46,rhino,1.00000
4,laramee,7806860-8295070-1223753-3922457,9/5/1287,m,vela garcia,0x2156f0e80x42ade1d0,legacy martin,1.00000
5,zelma,410412-14395340-16358919-6135465,3/11/1507,n,garlitz,0x2981558d0x5302ab1a,katempa,1.00000
6,rynale,11086872-12803247-3905009-16299420,3/11/1898,n,reuter,0x175401eb0x2ea803d6,javien,1.00000


### Performance Option / Seeding
Check here for potential for match.  If a match is impossible on available fields, can circumvent the linkage process and generate the MPI's here.

In [6]:
# Check for match availability.  If not, halt process and create MPIs
from mpi.link import is_match_available
from mpi.update import generate_mpi, write_mpi_data, gen_mpi_insert
from mpi.update import update_mpi_vector_table
from mpi.prepare import standardize


if is_match_available(dview.subset, iview):
    print('Match available.  Proceed with linking process.')
else:
    print('Match unavailable.  Generated MPIs for data view.')
    write_mpi_data(
        gen_mpi_insert(
            generate_mpi(standardize(dview.subset.drop_duplicates()))
        )
    )
    update_mpi_vector_table()
    
    # Recreate a view from the MPI table with valid identity data
    iview = create_identity_view(mapped_columns=dview.subset.columns)
    

checking available columns ['first_name_pool', 'birth_date_pool', 'gender_pool', 'last_name_pool', 'ssid_pool', 'middle_name_pool']
Match available.  Proceed with linking process.


## Building record linkage and mpi classification

In [7]:
from mpi.prepare import standardize
from utils import match_dtype

### Preprocessing

Standardize data across data and identity views.

In [8]:
# Match Dtypes - Align data types prior to cleaning.
#    This helps the cleaner by segmenting string/object and numeric fields

# Cast columns to matching datatypes for comparisons later on
source_clean, id_clean = match_dtype(standardize(dview.subset.drop_duplicates()), iview)

In [9]:
source_clean.head(1)

,birth_date_pool,first_name_pool,gender_pool,last_name_pool,middle_name_pool,ssid_pool,usbe_student_id_pool,guid
0,6/9/1256,genora,f,zegarra,rhino,0x218b2ea30x43165d46,666165,1489953705965142162


In [10]:
id_clean.head(1)

,first_name_pool,mpi,birth_date_pool,gender_pool,last_name_pool,ssid_pool,middle_name_pool,freq_score
0,None,1,None,None,None,None,None,0.57735


## Indexing

Make record pairs - pair rows needing match to potential identity candidates.

Indexing serves two purposes:

1. Create the list of pairs to check (candidate link).  Example: row 1 from table 1 to row 199 from table 2.

2. Reduce the potential number of pairs to check (candidates).

In [11]:
from mpi.index import build_indexer
from utils import match_dataframe_columns

In [12]:
# Create indexer on dataview
#    Indexer is a set of rules to generate 
#    candidate matches from data -> identities

source_matched, id_matched = match_dataframe_columns(source_clean, id_clean)

indexer = build_indexer(source_matched)

# Check index algorithms (generated from data view columns)
indexer.algorithms

[<SortedNeighbourhood left_on='first_name_pool', right_on='first_name_pool'>,
 <SortedNeighbourhood left_on='last_name_pool', right_on='last_name_pool'>,
 <SortedNeighbourhood left_on='middle_name_pool', right_on='middle_name_pool'>,
 <Block left_on='ssid_pool', right_on='ssid_pool'>]

In [13]:
# Run indexer on dataview, identity view
candidates = indexer.index(source_matched, id_matched)

# Full indexing is a cross join of data and all possible identities.

# Demonstrating full indexing size:
print('Full Index Length: ', len(source_clean) * len(id_clean))

# Examine multi indices.  On the left is the data view index.  Right identity.
print('Algorithmic Index Length: ', len(candidates))

# Estimate Savings
print('Savings: ', (1- len(candidates)/(len(source_clean) * len(id_clean))) * 100)

# Preview indices:
for pair in candidates[0:5]:
    print(f'Data-row {pair[0]}', f'ID-row {pair[1]}')

Full Index Length:  100010000
Algorithmic Index Length:  94600
Savings:  99.9054094590541
Data-row 0 ID-row 3
Data-row 0 ID-row 1710
Data-row 0 ID-row 4308
Data-row 0 ID-row 4983
Data-row 0 ID-row 6268


## Comparing

Indexing does not normally store the outcome of its findings.  Indexing algorithms are meant to be fast, can be error prone.  Algorithms can be tuned for string (many), numeric, and time/date fields.

The output of comparison is a clean feature matrix for the classifier to train/predict on.

In [14]:
from mpi.compare import build_comparator

In [15]:
# Create comparator on dataview
#    Comparator is a set of algorithms for each feature to be compared.
#    These are genearlly much more expensive compared to indexing functions
cmp = build_comparator(source_matched)

# Check comparison algorithms and fields
cmp.features

[<Exact 'ssid_pool'>,
 <String 'first_name_pool'>,
 <String 'gender_pool'>,
 <String 'last_name_pool'>,
 <String 'middle_name_pool'>]

In [16]:
# Compute comparisons
#    Gives clean match dataset for classification
comparisons = cmp.compute(candidates, source_clean, id_clean)
comparisons.head()

ssid_pool  first_name_pool  gender_pool  last_name_pool  \
0 3             1              1.0          1.0             1.0   
  1710          0              0.0          0.0             0.0   
  4308          0              0.0          0.0             0.0   
  4983          0              0.0          1.0             0.0   
  6268          0              0.0          1.0             0.0   

        middle_name_pool  
0 3                  1.0  
  1710               1.0  
  4308               0.0  
  4983               0.0  
  6268               0.0

## Classification

Score candidates for match.  

#### Two approaches: Supervised vs Unsupervised
 * **Supervised** approach requires a training set.
 * **Unsupervised** does not require a training set and operates on only on the comparison table itself.

In [17]:
from mpi.classify import estimate_true, build_classifier

# Get estimated true linkages for supervised model
links_true = estimate_true(comparisons)

# Create classifier
clf = build_classifier('logistic', comparisons, match_index=links_true)

# Check probabilities (score) of each comparison -- NOT IN USE IN THIS VERSION
predictions = clf.prob(comparison_vectors=comparisons)
predictions

0     3        9.991604e-01
      1710     2.174755e-05
      4308     3.915153e-07
      4983     2.631777e-06
      6268     2.631777e-06
                   ...     
9999  5478     1.319463e-04
      6681     2.444960e-04
      7863     2.631777e-06
      9755     2.631777e-06
      10002    9.991604e-01
Length: 94600, dtype: float64

## Evaluate
Express classification quality and explore outliers

In [18]:
from recordlinkage import reduction_ratio
from recordlinkage import confusion_matrix

links_pred = clf.predict(comparison_vectors=comparisons)

rratio = reduction_ratio(links_pred, source_clean)
cmatrix = confusion_matrix(links_true, links_pred, candidates)

In [19]:
# Review confusion matrix
#    TP-FN
#    |  |
#    FP-TN
print(cmatrix)

# Review reduction ratio
print(rratio)

[[ 9999     0]
 [    1 84600]]
0.9997999799979999


The confusion matrix may not be particularly useful here as generation of true links is prone to error. The reduction ratio is more sensitive than binary predictions in this case.

In [20]:
# Review findings
#   Interesting that the logistic predicted an MPI indices for each given an incomplete target list.

# Is the relationship 1,1?
split_list = lambda x: ([ix[0] for ix in x], [ix[1] for ix in x])
i1, i2 = split_list(links_pred)
len(list(set(i1))), len(list(set(i2)))

(10000, 10000)

## Update

Append matched MPIs and match score to data view and merge to original data.

In [27]:
from mpi.update import append_mpi, write_matched_unmatched
from utils import union_frames

# Attach matched MPIs and generate MPIs for unmatched rows
matched, unmatched = append_mpi(source_clean, id_clean, links_pred)

# Write matched and unmatched data to DB
write_matched_unmatched(matched, unmatched)

# Combine matched, unmatched
combined = union_frames(matched, unmatched)

# Merge mpi aligned data with standardized raw
smerge = dview.merge(combined, use_standardized=True)

# Update raw data with MPI
dview.update({'mpi':smerge.mpi})

In [30]:
# Update the MPI Vectors table for future use
from mpi.update import update_mpi_vector_table
update_mpi_vector_table()

### De-Identification

Create de-identified table while match available in memory or as referenced temp table.

In [31]:
from db import dataframe_to_db
from di import simple_di

dataframe_to_db(
    simple_di(dview.data), 
    tablename=source_tablename + '_di'
)

'usbe_students_di'

## Flag MPI

Rule 1:  MPI contains disagreement in blocking identifiers (local_id, ssn, ssid)

Rule 2:  Blocking identifer shared between multiple MPI

In [32]:
from mpi.postprocess import Rule1, Rule2

In [33]:
f = Rule1()
f.result

,flag


In [34]:
f = Rule2()
f.result

,flag
